In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score,recall_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from scipy.stats import norm
from tabpy.tabpy_tools.client import Client
from joblib import load

In [ ]:
# Deploy to Tableau
def deploy_tableau(title, function, description):
  client = Client('http://localhost:9004/')
  client.deploy(
    title,
    function,
    description,
    override=True
  )

In [2]:
# Import data
df = pd.read_csv("equipment_anomaly_data.csv")

In [9]:
model = load("model_ML.pkl")

In [10]:
# Function to calculate the probability to classify given values as 'Faulty'
def prob_faulty(_arg1,_arg2,_arg3,_arg4):
  """

  Args:
      _arg1 (float, list): Temperature
      _arg2 (float, list): Pressure
      _arg3 (float, list): Vibration
      _arg4 (float, list): Humidity

  Returns:
      _type_: _description_
  """
  input_data = np.column_stack([_arg1,_arg2,_arg3,_arg4])
  result = model.predict_proba(input_data)[0][1]
  return result.tolist()

In [11]:
deploy_tableau(
  "Probability_Faulty",
  prob_faulty,
  "Return the probability to classify given values as Faulty"
)

In [13]:
def norm_cdf(_arg1, _arg2, _arg3):
  """Return the Cumulative Distribution Function (CDF)

  Args:
      _arg1 (float): Pressure value_
      _arg2 (float): Average Pressure
      _arg3 (float): Std Pressure
  """
  prob = norm.cdf(_arg1, loc=_arg2, scale=_arg3)
  return prob.tolist()

In [15]:
# Deploy to Tableau
deploy_tableau('Probability_CDF', norm_cdf, 'Returns the probability CDF')

In [16]:
def norm_range(_arg1, _arg2, _arg3, _arg4):
  """Return the probability inside a range

  Args:
      _arg1 (float): Lower Pressure value_
      _arg2 (float): Upper Pressure value_
      _arg3 (float): Average Pressure
      _arg4 (float): Std Pressure
  """
  prob_lower = norm.cdf(_arg1, loc=_arg3, scale=_arg4)
  prob_upper = norm.cdf(_arg2, loc=_arg3, scale=_arg4)
  prob = prob_upper - prob_lower
  return prob.tolist()

In [18]:
def norm_pdf(_arg1, _arg2, _arg3):
  """Return the Probability Density Function (PDF)

  Args:
      _arg1 (float): Pressure value_
      _arg2 (float): Average Pressure
      _arg3 (float): Std Pressure
  """
  prob = norm.pdf(_arg1, loc=_arg2, scale=_arg3)
  return prob.tolist()